# Arxiv Dataset Clustering
This notebook aims to cluster scholarly articles from the Arxiv dataset.

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

## Load and Preview Data

In [ ]:
input_file = '../data/data_preprocessed.csv'
df = pd.read_csv(input_file)
df.head()

## Text Vectorization

In [ ]:
def vectorize_text(df):
    # Replace NaN values in the 'text' column with empty strings
    df['text'] = df['text'].fillna('')
    # Instantiate TfidfVectorizer
    vectorizer = TfidfVectorizer(max_features=1000)
    X = vectorizer.fit_transform(df['text'])
    return X, vectorizer

tfidf_matrix, vectorizer = vectorize_text(df)

In [ ]:
# Sample a fraction of data (e.g., 10%)
sample_fraction = 0.1
df_sample = df.sample(frac=sample_fraction)

# Vectorize the sampled text data:
vectorizer = TfidfVectorizer(norm="l2", analyzer="word", ngram_range=(2,3), min_df=5, max_df=0.99, max_features=1500, lowercase=False)
tfidf_vectors = vectorizer.fit_transform(df_sample['text'])
tfidf_words = vectorizer.get_feature_names_out()

# Looking for most frequent words
sums = tfidf_vectors.sum(axis=0)
data = []

# For each column, get the tuple (word, frequency)
for col, word in enumerate(tfidf_words):
    data.append((word, sums[0,col]))
frequency = pd.DataFrame(data, columns=['words', 'freq'])

print("\nNumber of words:\n", frequency.sort_values('freq', ascending=False)[:20])
print(f"Data samples: {tfidf_vectors.shape[0]}")


## Dimensionality Reduction

In [ ]:
def reduce_dimensionality(X):
    # Instantiate SVD
    svd = TruncatedSVD(n_components=100, random_state=42)
    # Fit SVD to the data
    svd.fit(X)
    # Apply transform to the data
    X_svd = svd.transform(X)
    return X_svd

X_svd = reduce_dimensionality(tfidf_matrix)


## Visual Inspection using t-SNE

In [ ]:
def visualize_tsne(X):
    # Taking a random sample of 1% of the data
    sample_indices = np.random.choice(X.shape[0], int(0.1 * X.shape[0]), replace=False)
    X_sample = X[sample_indices]

    # Instantiating and fitting t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_sample)

    # Plotting the t-SNE features in a scatter plot and coloring neighbor clusters differently
    plt.figure(figsize=(12, 8))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], marker='.' , alpha=0.6)
    plt.title('Data visualization using t-SNE')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()


visualize_tsne(X_svd)

## Determine Optimal Clusters

In [ ]:
# calculate the sum of squared distances for several k and plot the results:
fig = plt.figure(figsize=(8,8))
within_cluster_squares={}
for i in range(3,10):
    kmeans = KMeans(n_clusters=i, n_init=10, random_state=42).fit(X_svd)
    within_cluster_squares[i] = kmeans.inertia_
plt.plot(list(within_cluster_squares.keys()),list(within_cluster_squares.values()))
plt.xlabel('Values for K')
plt.ylabel('Sum of Squared Distances')
plt.grid(False)

plt.show()

In [ ]:
def determine_optimal_clusters(X):
    wcss = []
    silhouette_scores = []
    davies_bouldin_scores = []


    # Taking a random sample of 1% of the data
    sample_indices = np.random.choice(X.shape[0], int(0.1 * X.shape[0]), replace=False)
    X_sample = X[sample_indices]

    # Define a range of clusters to test
    cluster_range = range(2, 11)

    for i in cluster_range:
        kmeans = KMeans(n_clusters=i, n_init=10, random_state=0)
        kmeans.fit(X_sample)
        wcss.append(kmeans.inertia_)

        # Calculate silhouette score
        silhouette_avg = silhouette_score(X_sample, kmeans.labels_)
        silhouette_scores.append(silhouette_avg)


        # Calculate Davies-Bouldin index
        davies_bouldin_avg = davies_bouldin_score(X_sample, kmeans.labels_)
        davies_bouldin_scores.append(davies_bouldin_avg)

        # Print the score for each cluster
        print("For n_clusters = {}, silhouette score is {})".format(i, silhouette_avg))
        print("For n_clusters = {}, Davies-Bouldin score is {})".format(i, davies_bouldin_avg))


    # Plot the scores 
    plt.figure(figsize=(15, 5))

    # Plot Silhouette Score
    plt.subplot(1, 2, 1)
    plt.plot(cluster_range, silhouette_scores, marker='o', linestyle='-', color='g')
    plt.title('Silhouette Score')
    plt.xlabel('Number of clusters')
    plt.ylabel('Score')

    # Plot Davies-Bouldin Score
    plt.subplot(1, 2, 2)
    plt.plot(cluster_range, davies_bouldin_scores, marker='o', linestyle='-', color='r')
    plt.title('Davies-Bouldin Score')
    plt.xlabel('Number of clusters')
    plt.ylabel('Score')
    
    plt.show()

# Call the function to determine optimal clusters
determine_optimal_clusters(X_svd)

## Apply Clustering

In [ ]:
def apply_clustering(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    clusters = kmeans.fit_predict(X)
    return clusters

optimal_clusters = 8  # Adjusted based on observation.
df['cluster'] = apply_clustering(X_svd, optimal_clusters)

In [ ]:
# print each cluster most frequent words in a dataframe and show a table
def print_cluster_words(df, vectorizer, cluster_number, top_n_words):
    # Filter the dataframe to only get the rows associated with the given cluster
    cluster_df = df[df['cluster'] == cluster_number]

    # Get the text data from the dataframe
    text_data = cluster_df['text']

    # Instantiate the vectorizer
    vectorizer = vectorizer

    # Fit and transform the vectorizer on the text data
    vectorized_data = vectorizer.fit_transform(text_data)

    # Create a list of the vectorized words
    vectorized_data_as_array = vectorized_data.toarray()

    # Create a DataFrame with the words
    vocab = vectorizer.get_feature_names()
    word_counts = pd.DataFrame({'word': vocab, 'count': np.sum(vectorized_data_as_array, axis=0)})
    word_counts = word_counts.sort_values(by='count', ascending=False)


    # Print the top n words from the DataFrame
    print(word_counts.sort_values(by='count', ascending=False).head(top_n_words))

    # Create and generate a word cloud image
    wordcloud = WordCloud().generate(' '.join(vocab))

    # Display the generated image:
    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()


for i in range(optimal_clusters):
    print_cluster_words(df, vectorizer, i, 10)


## Visualizing Clusters

In [ ]:
def plot_clusters(df):
    plt.figure(figsize=(10,6))
    sns.countplot(x='cluster', data=df)
    plt.title('Distribution of Clusters')
    plt.xlabel('Cluster')
    plt.ylabel('Number of Samples')
    plt.show()

plot_clusters(df)

def visualize_clusters(X, clusters):
    # Taking a random sample of 1% of the data
    sample_indices = np.random.choice(X.shape[0], int(0.01 * X.shape[0]), replace=False)
    X_sample = X[sample_indices]
    clusters_sample = clusters[sample_indices]

    # Instantiating and fitting t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_sample)

    # Plotting the t-SNE features
    plt.figure(figsize=(12, 8))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], marker='.', c=clusters_sample, cmap='viridis')
    plt.title('Data visualization using t-SNE')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()

    # plotting in 3D

    # Instantiating and fitting t-SNE
    tsne = TSNE(n_components=3, random_state=42)
    X_tsne = tsne.fit_transform(X_sample)

    # Plotting the t-SNE features
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], marker='.', c=clusters_sample, cmap='viridis')
    plt.title('Data visualization using t-SNE')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()


visualize_clusters(X_svd, df['cluster'])

## Save Results

In [ ]:
output_file = '../data/data_clustered.csv'
df.to_csv(output_file, index=False)
